## [ 결측치 Missing Value 처리 ]
- 데이터 미입력 또는 데이터 저장 과정에서 지워진 데이터
- 빈칸 의미
- 표시 : NaN (Not A Number) NaT(Not a Text)
- 표현 : numpy 모듈의 nan, math 모듈의 nan 사용

(1) 모듈 로딩

In [71]:
import pandas as pd

(2) 데이터 준비

In [72]:
file = '../DATA/employees.csv'

(3) 데이터 저장 : CSV => DataFrame

원본 파일 열어서 봐야할거
- 첫줄이 데이터인지 컬럼인지 확인
- 결측치 존재 유무
- 열 중에서 행 인덱스로 쓰일만한게 있는지 확인 (빈칸이나 중복이 있으면 안됨)

In [73]:
# 구분자: ,           헤더: 첫번째 줄 컬럼명
empDF=pd.read_csv(file)

(4) 데이터 확인

In [74]:
# 데이터의 전반적인 요약정보 확인 => info() 메서드
# - 컬럼명 결측치 여부 확인 ==> 실제 데이터에서 결측치 체크 & 처리
# - 컬럼별 데이터 타입과 실제 데이터 타입 비교 ==> 타입 다른 경우, 타입 변환
empDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   First Name  933 non-null    object 
 1   Gender      854 non-null    object 
 2   Start Date  999 non-null    object 
 3   Salary      999 non-null    float64
 4   Mgmt        933 non-null    object 
 5   Team        957 non-null    object 
dtypes: float64(1), object(5)
memory usage: 47.0+ KB


- 데이터의 컬럼별 분포 확인 => describe() : 컬럼별 기술통계값

In [75]:
empDF.describe()

,Salary
count,999.000000
mean,90655.528529
std,32939.511615
min,35013.000000
25%,62560.000000
50%,90427.000000
75%,118744.500000
max,149908.000000


In [76]:
# 최빈값 => mode(), 중앙값 => median()
print(empDF.mode(numeric_only = True),
      empDF.median(numeric_only=True),
      empDF.mean(numeric_only=True),
      sep='\n\n')

     Salary
0   86676.0
1   91462.0
2  121160.0
3  145988.0
4  147183.0

Salary    90427.0
dtype: float64

Salary    90655.528529
dtype: float64


(5) 데이터 전처리(Preprocessing) => 가. 결측치 처리

- 결측치 확인 ==> isna(), isnull() : 원소 단위로 검사 후 결측치인 경우 True 반환

In [77]:
# 전체 데이터의 컬럼별 결측치 체크
print(empDF.isna().head(), empDF.isnull().head(), sep = '\n\n')

   First Name  Gender  Start Date  Salary   Mgmt   Team
0       False   False       False    True  False  False
1       False   False       False   False  False   True
2       False   False        True   False  False  False
3       False    True       False   False  False  False
4       False   False       False   False  False  False

   First Name  Gender  Start Date  Salary   Mgmt   Team
0       False   False       False    True  False  False
1       False   False       False   False  False   True
2       False   False        True   False  False  False
3       False    True       False   False  False  False
4       False   False       False   False  False  False


In [78]:
# 전체 데이터의 컬럼별 결측치 체크 후 갯수 파악 => 합계
print(empDF.isna().sum(), empDF.isnull().sum(), sep = '\n\n')

First Name     68
Gender        147
Start Date      2
Salary          2
Mgmt           68
Team           44
dtype: int64

First Name     68
Gender        147
Start Date      2
Salary          2
Mgmt           68
Team           44
dtype: int64


In [79]:
empDF.notna()

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,True,True,True,False,True,True
1,True,True,True,True,True,False
2,True,True,False,True,True,True
3,True,False,True,True,True,True
4,True,True,True,True,True,True
...,...,...,...,...,...,...
996,True,True,True,True,True,True
997,True,True,True,True,True,True
998,True,True,True,True,True,True
999,True,True,True,True,True,True


- 결측치 확인 ==> notna() / notnull()  : 결측치가 아니면 True 반환

In [80]:
print(empDF.notna().head(), empDF.notnull().head(), sep='\n\n')

   First Name  Gender  Start Date  Salary  Mgmt   Team
0        True    True        True   False  True   True
1        True    True        True    True  True  False
2        True    True       False    True  True   True
3        True   False        True    True  True   True
4        True    True        True    True  True   True

   First Name  Gender  Start Date  Salary  Mgmt   Team
0        True    True        True   False  True   True
1        True    True        True    True  True  False
2        True    True       False    True  True   True
3        True   False        True    True  True   True
4        True    True        True    True  True   True


In [81]:
print(empDF.notna().sum(), empDF.notnull().sum(), sep='\n\n')

First Name    933
Gender        854
Start Date    999
Salary        999
Mgmt          933
Team          957
dtype: int64

First Name    933
Gender        854
Start Date    999
Salary        999
Mgmt          933
Team          957
dtype: int64


- 결측치 처리 ==> (1) 삭제 : dropna()

In [82]:
# [기본] 모든 행에 1개의 값이라도 NaN이면 삭제
empDF2 = empDF.dropna()            #복사본을 저장함
print(empDF.shape, empDF2.shape)

(1001, 6) (761, 6)


In [83]:
# [설정] 행에 모든 데이터 즉 값이 NA면 삭제
# 즉, 1개의 행을 뽑았을 때 모든 열이 NaN이면 삭제하겠다. 하나라도 데이터가 있다면 삭제는 ㄴㄴ

empDF3 = empDF.dropna(how = 'all')
print(empDF.shape, empDF3.shape)

(1001, 6) (1000, 6)


In [84]:
# [설정] 특정 컬럼의 NA만 체크 후 삭제하는 방법
# 즉, Salary에 대해 분석하려는데 어떤 행이 Salary 빼고 다른 열은 데이터가 있다한들 필요없으니 삭제하겠다.

# 예) 성별에 따른 연봉 분석 => NA면 분석 불가능 컬럼을 지정
empDF3 = empDF.dropna(subset = ['Gender', 'Salary'])
print(empDF.shape, empDF3.shape)

(1001, 6) (853, 6)


In [85]:
# [설정] 컬럼별 정상데이터 즉 NA가 아닌 데이터의 개수를 지정 후 해당 수만큼 정상데이터가 없는 컬럼 삭제
# 즉, 열 안에서 정상데이터가 몇개 이하면 그 열을 삭제시켜라

empDF5 = empDF.dropna(thresh = 900, axis='columns')
print(empDF.shape, empDF5.shape, empDF5.columns)

(1001, 6) (1001, 5) Index(['First Name', 'Start Date', 'Salary', 'Mgmt', 'Team'], dtype='object')


- 결측치 처리 => 대체 : fillna()

In [86]:
# Gender 컬럼에 결측값을 무엇으로 채울지? 결졍해보자
empDF.describe(include = 'all').head(4)

,First Name,Gender,Start Date,Salary,Mgmt,Team
count,933,854,999,999.0,933,957
unique,200,2,971,NaN,2,10
top,Marilyn,Female,10/30/04,NaN,True,IT
freq,11,431,2,NaN,468,106


In [87]:
# ==> 컬럼의 최빈값으로 na 채우기
empDF.Gender.mode()[0]           #mode까지 하면 시리즈가 나오니까 최빈값을 인덱스로 접근
genSR = empDF.Gender.fillna(empDF.Gender.mode()[0])    #시리즈에 접근한거라 시리즈 원본에 바꿔줌
genSR.isna().sum()               # 결측값이 사라진걸 알수있음

0

In [88]:
# ==> NA값 이전과 이후 값으로 채우기 => method = 'ffill', method = 'bfill'

# 결측값을 앞의 값(그전 인덱스 값)으로 채우자
empDF.Gender.fillna(method = 'ffill') 
# 이 시리즈에서는 bfill하면 마지막 값이 NaN이 됨(채울게 없음)

#empDF.Gender.fillna(method = 'ffill', limit =  1) 
#이르케 하면 하나만 채우겠다.

0         Male
1         Male
2       Female
3       Female
4         Male
         ...  
996       Male
997       Male
998       Male
999       Male
1000      Male
Name: Gender, Length: 1001, dtype: object

In [89]:
import numpy as np
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                    [3, 4, np.nan, 1],
                    [np.nan, np.nan, np.nan, np.nan],
                    [np.nan, 3, np.nan, 4]],
                   columns=list("ABCD"))
df

,A,B,C,D
0,NaN,2.0,NaN,0.0
1,3.0,4.0,NaN,1.0
2,NaN,NaN,NaN,NaN
3,NaN,3.0,NaN,4.0


In [90]:
df.A.fillna(method = 'bfill')

0    3.0
1    3.0
2    NaN
3    NaN
Name: A, dtype: float64

In [91]:
df.A.fillna(method = 'ffill', limit = 1)         #여기서 limit은 최대 채우는 개수를 지정함

0    NaN
1    3.0
2    3.0
3    NaN
Name: A, dtype: float64

In [92]:
df2 = pd.DataFrame(np.zeros((4,4)), columns = list("ABCE"))
df.fillna(df2)   #여기서는 E열이 없기때문에 D는 결측값이 채워지지 않는다. (일대일대응이니까)

,A,B,C,D
0,0.0,2.0,0.0,0.0
1,3.0,4.0,0.0,1.0
2,0.0,0.0,0.0,NaN
3,0.0,3.0,0.0,4.0


In [93]:
# B 컬럼과 D 컬럼을 평균으로 결측치를 대체하고자 함
df.B.fillna(df.B.mean())

0    2.0
1    4.0
2    3.0
3    3.0
Name: B, dtype: float64

In [94]:
df.D.fillna(df.D.mean())

0    0.000000
1    1.000000
2    1.666667
3    4.000000
Name: D, dtype: float64